Datenterm: $$\sum_{(i,j)\in k}(r_{ij}-\vec{p}_i^T\vec{q}_j)^2$$
Matrixdarstellung: $$||R-P Q^T||_2^2$$

In [1]:
import numpy as np

def cost(P, Q, R, lambda_, np, nq):
    data_term = np.linalg.norm(R - P @ Q.T)**2
    regularization_p = np.sum(np * np.linalg.norm(P)**2)
    regularization_q = np.sum(nq * np.linalg.norm(Q)**2)
    regularization_term = lambda_ * (regularization_p + regularization_q)
    return data_term + regularization_term

In [2]:
def solveBlock_P(R, Q, I, lambda_, n_p):
    m, n = R.shape
    k = Q.shape[1]
    P = np.zeros((m, k))
    I = I.astype(bool)

    for i in range(m):
        I_i = I[i, :]
        Q_i = Q[I_i, :]
        R_i = R[i, I_i]

        A = Q_i.T @ Q_i + lambda_ * n_p[i] * np.eye(k)
        b = Q_i.T @ R_i
        P[i, :] = np.linalg.solve(A, b)
    return P

def solveBlock_Q(R, P, I, lambda_, n_q):
    m, n = R.shape
    k = P.shape[1]
    Q = np.zeros((n, k))
    I = I.astype(bool)

    for j in range(n):
        I_j = I[:, j]
        P_j = P[I_j, :]
        R_j = R[I_j, j]

        A = P_j.T @ P_j + lambda_ * n_q[j] * np.eye(k)
        b = P_j.T @ R_j
        Q[j, :] = np.linalg.solve(A, b)
    return Q

In [3]:
def calc_rmse(R, I, P, Q):
    R_pred = P @ Q.T
    error = I * (R - R_pred)
    rmse = np.sqrt(np.sum(error**2) / np.sum(I))
    return rmse

def alternating_least_squares(R, I, k=3, lambda_=0.1, max_iter=100, tolerance=1e-4):
    m, n = R.shape
    P = np.random.rand(m, k)
    Q = np.random.rand(n, k)
    n_p = np.ones(m)
    n_q = np.ones(n)
    rmse_history = []

    for iteration in range(max_iter):
        P = solveBlock_P(R, Q, I, lambda_, n_p)
        Q = solveBlock_Q(R, P, I, lambda_, n_q)
        
        rmse = calc_rmse(R, I, P, Q)
        rmse_history.append(rmse)
        
        print(f"Iteration {iteration + 1}/{max_iter}, RMSE: {rmse:.6f}")
        
        if iteration > 0 and abs(rmse_history[-1] - rmse_history[-2]) < tolerance:
            print("convergence reached")
            break

    return P, Q, rmse_history

In [4]:
from utility import setup_dataset, visualize_sparsity_pattern

R, I = setup_dataset(toy_ds=True)
P, Q, rmse_history = alternating_least_squares(R, I, k=3, lambda_=0.1, max_iter=50)
P, Q

Iteration 1/50, RMSE: 0.027936
Iteration 2/50, RMSE: 0.034818
Iteration 3/50, RMSE: 0.033279
Iteration 4/50, RMSE: 0.033273
convergence reached


(array([[ 2.1930279 ,  1.44309202,  4.48367353],
        [ 3.31692156,  2.15350214,  4.26301863],
        [ 0.21864235,  4.04081638, -0.48142184],
        [ 0.89907871,  3.26572723, -0.17479528]]),
 array([[ 0.03371364,  0.35578269,  0.9732352 ],
        [ 0.81607216,  0.11124708,  0.46767288],
        [ 0.03641664,  1.18586816, -0.39122294]]))

In [5]:
R, I = setup_dataset(toy_ds=False)
P, Q, rmse_history = alternating_least_squares(R, I, k=10, lambda_=0.1, max_iter=100)

Iteration 1/100, RMSE: 0.833264
Iteration 2/100, RMSE: 0.754412
Iteration 3/100, RMSE: 0.724516
Iteration 4/100, RMSE: 0.707673
Iteration 5/100, RMSE: 0.697696
Iteration 6/100, RMSE: 0.690939
Iteration 7/100, RMSE: 0.685797
Iteration 8/100, RMSE: 0.681821
Iteration 9/100, RMSE: 0.678780
Iteration 10/100, RMSE: 0.676427
Iteration 11/100, RMSE: 0.674560
Iteration 12/100, RMSE: 0.673036
Iteration 13/100, RMSE: 0.671764
Iteration 14/100, RMSE: 0.670683
Iteration 15/100, RMSE: 0.669738
Iteration 16/100, RMSE: 0.668900
Iteration 17/100, RMSE: 0.668155
Iteration 18/100, RMSE: 0.667498
Iteration 19/100, RMSE: 0.666919
Iteration 20/100, RMSE: 0.666408
Iteration 21/100, RMSE: 0.665953
Iteration 22/100, RMSE: 0.665544
Iteration 23/100, RMSE: 0.665175
Iteration 24/100, RMSE: 0.664840
Iteration 25/100, RMSE: 0.664532
Iteration 26/100, RMSE: 0.664249
Iteration 27/100, RMSE: 0.663988
Iteration 28/100, RMSE: 0.663745
Iteration 29/100, RMSE: 0.663519
Iteration 30/100, RMSE: 0.663307
Iteration 31/100, R